## Marker notebook 

In [2]:
import numpy as np

# Paramètres de la tâche (valeurs fixes)
cornerX = 302
cornerY = 80
centerX = 552
centerY = 330
externalRadius = 250
internalRadius = 170
borderRadius = 1
cursorRadius = 16
indexOfDifficulty = 28.00696429476858
taskRadius = 209.5
taskTolerance = 47
cycleMaxNumber = 6
cycleDuration = 20


data = np.loadtxt("/Users/matysprecloux/Desktop/Master IEAP/IEAP-project/Circular-task-analysis:/Data/001MoDe_R1.csv",
                  delimiter=',',skiprows=1)

# skiprows=1 -> on saute l’entête contenant les noms de colonnes

timestamps_ms = data[:,0]
mouseX = data[:,1]
mouseY = data[:,2]
mouseInTarget = data[:,3]

# Recentrer le repère
x_centered = mouseX - centerX
y_centered = mouseY - centerY
dist = np.sqrt(x_centered**2 + y_centered**2)

# Convertir en secondes depuis le début du fichier
t0 = timestamps_ms[0]
timestamps = (timestamps_ms - t0) / 1000.0

# Définir les temps de début et de fin des enregistrements (en secondes)
record_intervals = [
    (0.022, 20.004),
    (40.429, 60.018),
    (80.371, 100.050),
    (120.608, 140.069),
    (160.146, 180.098)
]

# inside = 1 si DANS l’anneau [internalRadius, externalRadius]
is_inside_csv = (dist >= internalRadius) & (dist <= externalRadius)
flags = is_inside_csv.astype(int)
record_times = record_intervals

def extract_record(times, x, y, inTarget, start, end):
    """
    Extrait un record selon les start/stop définis en secondes.
    """
    mask = (times >= start) & (times <= end)

    return (
        times[mask],
        x[mask],
        y[mask],
        inTarget[mask]
    )
all_records = []

for (start, end) in record_times:
    rec_t, rec_x, rec_y, rec_in = extract_record(
        timestamps, mouseX, mouseY, mouseInTarget,
        start, end
    )
    all_records.append((rec_t, rec_x, rec_y, rec_in))

# ----- Step 1 : Remove everything before first entry into target -----
first_inside_indices = np.where(flags == 1)[0]
if len(first_inside_indices) > 0:
    first_idx = first_inside_indices[0]
    # garder seulement la vraie phase circulaire
    t = timestamps[first_idx:]
    x = mouseX[first_idx:]
    y = mouseY[first_idx:]
    flags = flags[first_idx:]
    

In [3]:

for i, (start, end) in enumerate(record_intervals):

    mask = (timestamps >= start) & (timestamps <= end)

    t_all = timestamps[mask]
    x_all = x_centered[mask]
    y_all = y_centered[mask] 
    dist_all = dist[mask]
    inside_all = is_inside_csv[mask]

    if len(t_all) == 0:
        print(f"Rec{i+1:03d} ,   0.00 ,     0.00 ,     0.00 ,    nan ,    nan ,    nan ,    0.00 ,    0.00")
        continue

    
    # STEP 1 — keep only data from first entry into the target
    
    first_inside_idx = np.where(inside_all)[0]
    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_all = t_all[fi:]
        x_all = x_all[fi:]
        y_all = y_all[fi:]
        dist_all = dist_all[fi:]
        inside_all = inside_all[fi:]


    # STEP 2 - nLaps with first inside idx 
def compute_radius_angle(x, y, centerX, centerY):
    r = np.sqrt((x - centerX)**2 + (y - centerY)**2)
    theta = np.arctan2(y - centerY, x - centerX)
    return r, theta

def compute_nLaps(theta):
    theta_unwrapped = np.unwrap(theta)
    delta_theta = theta_unwrapped[-1] - theta_unwrapped[0]
    nLaps = delta_theta / (2 * np.pi)
    return nLaps

# ---- VERSION QUI UTILISE FIRST ENTRY INTO TARGET ----

for i, (t, x, y, inp) in enumerate(all_records):

    # STEP 1 — keep only samples AFTER first entry into target
    inside = (inp == 1)  # inp = colonne mouseInTarget
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t = t[fi:]
        x = x[fi:]
        y = y[fi:]
        inp = inp[fi:]
        inside = inside[fi:]

    # STEP 2 — compute radius + angle
    r, theta = compute_radius_angle(x, y, centerX, centerY)

    # STEP 3 — compute nLaps
    nLaps = compute_nLaps(theta)

    print(f"Record {i} : {nLaps:.2f} tours")

Record 0 : 10.84 tours
Record 1 : 11.97 tours
Record 2 : 13.26 tours
Record 3 : 12.83 tours
Record 4 : 14.74 tours


In [4]:
# STEP 3 — Re = mean radius
for i, (t, x, y, inp) in enumerate(all_records):

    # STEP 1 — garder seulement après première entrée dans la cible
    inside = (inp == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t = t[fi:]
        x = x[fi:]
        y = y[fi:]
        inp = inp[fi:]
        inside = inside[fi:]

    # STEP 2 — recalculer la distance au centre
    dist = np.sqrt((x - centerX)**2 + (y - centerY)**2)

    # STEP 3 — Re
    Re = np.mean(dist)

    print(f"Record {i} : Re = {Re:.2f} px")

Record 0 : Re = 213.86 px
Record 1 : Re = 209.85 px
Record 2 : Re = 209.43 px
Record 3 : Re = 209.46 px
Record 4 : Re = 210.89 px


In [5]:

def compute_Te(x, y, centerX, centerY):
    """
    Calcule Te selon ta formule :
    Te = sigma * sqrt(2*pi*e)
    """
    # rayon r(t)
    r = np.sqrt((x - centerX)**2 + (y - centerY)**2)

    # sigma du rayon
    sigma = np.std(r)

    # constante sqrt(2*pi*e)
    K = np.sqrt(2 * np.pi * np.e)

    # Te final
    Te = sigma * K

    return Te, sigma, r


# -------- VERSION AVEC FILTRE FIRST INSIDE ----------
for i, (t_rec, x_rec, y_rec, in_rec) in enumerate(all_records):

    # STEP 1 — garder uniquement après première entrée dans la cible
    inside = (in_rec == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_rec = t_rec[fi:]
        x_rec = x_rec[fi:]
        y_rec = y_rec[fi:]
        in_rec = in_rec[fi:]
        inside = inside[fi:]

    # STEP 2 — calcul de Te selon TA formule
    Te, sigma, r = compute_Te(x_rec, y_rec, centerX, centerY)

    print(f"Record {i} : Te = {Te:.2f} px   (sigma = {sigma:.2f})")

Record 0 : Te = 91.78 px   (sigma = 22.21)
Record 1 : Te = 34.45 px   (sigma = 8.34)
Record 2 : Te = 42.42 px   (sigma = 10.26)
Record 3 : Te = 37.14 px   (sigma = 8.99)
Record 4 : Te = 43.88 px   (sigma = 10.62)


In [ ]:
def compute_IDe(Re, Te):
    return (2 * np.pi * Re) / Te


for i, (t_rec, x_rec, y_rec, in_rec) in enumerate(all_records):

    
    # STEP 1 — keep only data from first entry into the target
    
    inside = (in_rec == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_rec = t_rec[fi:]
        x_rec = x_rec[fi:]
        y_rec = y_rec[fi:]
        in_rec = in_rec[fi:]
        inside = inside[fi:]

    
    # STEP 2 — compute Te using YOUR formula
    
    Te, sigma, r = compute_Te(x_rec, y_rec, centerX, centerY)

    # ⚠️ ATTENTION :
    # compute_Te retourne (Te, sigma, r)
    # mais TA fonction attend (Re, Te)
    # donc ici on doit recalculer Re = mean(r)
    Re = np.mean(r)

    
    # STEP 3 — compute IDe
    
    IDe = compute_IDe(Re, Te)

    print(f"Record {i} : IDe/Lap = {IDe:.2f}")

Record 0 : IDe/Lap = 14.64
Record 1 : IDe/Lap = 38.27
Record 2 : IDe/Lap = 31.02
Record 3 : IDe/Lap = 35.44
Record 4 : IDe/Lap = 30.20


In [ ]:
# MT per lap

def compute_MT_per_lap(t, nLaps):
    if len(t) < 2 or nLaps == 0:
        return np.nan
    duration = t[-1] - t[0]
    return duration / abs(nLaps)



# LOOP over all 5 records

for i, (t_rec, x_rec, y_rec, in_rec) in enumerate(all_records):

    
    # STEP 1 — keep only samples AFTER first entry into target
    
    inside = (in_rec == 1)
    first_inside_idx = np.where(inside)[0]

    if len(first_inside_idx) > 0:
        fi = first_inside_idx[0]
        t_f = t_rec[fi:]
        x_f = x_rec[fi:]
        y_f = y_rec[fi:]
        in_f = in_rec[fi:]
    else:
        t_f = t_rec
        x_f = x_rec
        y_f = y_rec
        in_f = in_rec

    
    # STEP 2 — compute theta and nLaps
    
    theta = np.arctan2(y_f - centerY, x_f - centerX)
    nLaps = compute_nLaps(theta)

    
    # STEP 3 — compute MT/lap
    
    MT_lap = compute_MT_per_lap(t_f, nLaps)

    print(f"Record {i+1} : MT/lap = {MT_lap:.3f} s/lap")